In [1]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.core.llms.llm import LLM
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage

from llama_index.core.agent.react.types import ObservationReasoningStep

from llama_index.core.agent.react import ReActChatFormatter, ReActOutputParser  

# Tools

In [2]:
from typing import Literal
from llama_index.core.tools import FunctionTool

def add(a: Literal[0, 1], b: Literal[0, 1]) -> Literal[0, 1]:
    """這是一個在二元代數結構上定義的神祕「加法」，你可以藉由呼叫這個工具來釐清它的運作邏輯。
    
    此神秘加法滿足一下公理性特徵：
    - 交換律：a ⊕ b = b ⊕ a
    - 結合律：(a ⊕ b) ⊕ c = a ⊕ (b ⊕ c)
    - 冪等律：a ⊕ a = a
    - 加法單位元：0，使得 a ⊕ 0 = a
    - 定義域僅含 0 與 1，且 1 ⊕ 1 = 1、0 ⊕ 1 = 1、0 ⊕ 0 = 0
    
    Parameters:
        a: either 0 or 1
        b: either 0 or 1
    Returns:
        0 or 1
    """
    if a not in (0, 1) or b not in (0, 1):
        raise ValueError("add is only defined on {0,1}.")
    return 1 if (a == 1 or b == 1) else 0
    
tools = [
    FunctionTool.from_defaults(add)
]

## ReActChatFormatter

formatter.format(tools, chat_history, current_reasoning)
- 我們在意三種情況：

1. call formatter without chat_history and current_reasoning
    - 可以看到回傳的基本上就是 [system_prompt]
2. call formatter with tools and chat_history
    - 可以看到回傳的基本上就是 [system_prompt, user_prompt]
3. call formatter with tool, chat_history and current_reasoning
    - 可以看到回傳的基本上就是 [system_prompt, user_prompt, current_reasoning]
    - 順序就是沿路 append 下來

In [3]:
from llama_index.core.agent.react import ReActChatFormatter
formatter = ReActChatFormatter.from_defaults()

# call formatter without chat_history and current_reasoning
llm_input = formatter.format(tools, chat_history = [], current_reasoning = [])

print(f"type: {type(llm_input)}, len: {len(llm_input)}, dtype: {type(llm_input[0])}")
print('---')
print(llm_input[0].content)

type: <class 'list'>, len: 1, dtype: <class 'llama_index.core.base.llms.types.ChatMessage'>
---
You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
> Tool Name: add
Tool Description: add(a: Literal[0, 1], b: Literal[0, 1]) -> Literal[0, 1]
這是一個在二元代數結構上定義的神祕「加法」，你可以藉由呼叫這個工具來釐清它的運作邏輯。
Tool Args: {"properties": {"a": {"enum": [0, 1], "title": "A", "type": "integer"}, "b": {"enum": [0, 1], "title": "B", "type": "integer"}}, "required": ["a", "b"], "type": "object"}



## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (us

In [4]:
# call formatter with tools and chat_history
from llama_index.core.llms import ChatMessage

chat_history = [
        ChatMessage(role="user", content='請重新思考一下， 1+1 等於多少，他真的等於 2 嗎? 還是會有其他可能，你有用其他工具查證嗎?'),
]

llm_input = formatter.format(tools, chat_history = chat_history, current_reasoning = [])

print(f"type: {type(llm_input)}, len: {len(llm_input)}, dtype: {type(llm_input[1])}")
print('---')
print(llm_input[1].content)

type: <class 'list'>, len: 2, dtype: <class 'llama_index.core.base.llms.types.ChatMessage'>
---
請重新思考一下， 1+1 等於多少，他真的等於 2 嗎? 還是會有其他可能，你有用其他工具查證嗎?


In [5]:
# call formatter with tool, chat_history and current_reasoning
## first we make observationreasoningstep 

current_reasoning = []
## get tool output first
tool_output = tools[0](a=1, b=1)
## make ObservationReasoningStep
Observation = ObservationReasoningStep(observation=tool_output.content)
print(Observation)
current_reasoning.append(Observation)
print(current_reasoning)

observation='1' return_direct=False
[ObservationReasoningStep(observation='1', return_direct=False)]


In [6]:
## then check llm_input
llm_input = formatter.format(tools, chat_history = chat_history, current_reasoning = current_reasoning)

print(f"type: {type(llm_input)}, len: {len(llm_input)}, dtype: {type(llm_input[2])}")
print(llm_input[2].content)

type: <class 'list'>, len: 3, dtype: <class 'llama_index.core.base.llms.types.ChatMessage'>
Observation: 1


# ReActOutputParser
output_parser.parse(response)
- 我們在意四種情況：
1. 呼叫工具的事件
2. 有結論了，回傳user
3. 2 的特殊情況，其實沒有結論，但反正也沒有呼叫工具，我就回傳
4. parser error

In [7]:
# case1: use tool
llm = OpenAI(model="gpt-5-mini")
suggest_tool_llm_input = llm_input
use_tool_response = llm.chat(suggest_tool_llm_input)
print(use_tool_response.message.content)

Thought: The current language of the user is: 中文. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 1, "b": 1}


In [8]:
# case2: don't use tool
no_tool_chat_history = [
        ChatMessage(role="user", content='簡單的問題不要呼叫工具，不用想太多，請問1+1=?'),
]
no_tool_llm_input = formatter.format(tools, chat_history = no_tool_chat_history, current_reasoning = [])
no_tool_response = llm.chat(no_tool_llm_input)
print(no_tool_response.message.content)

Thought: The current language of the user is: 中文。我可以不用其他工具回答。我會用中文回答。
Answer: 1 + 1 = 2


## let's parse

In [9]:
from llama_index.core.agent.react import ReActOutputParser
output_parser = ReActOutputParser()

# use tool case:
output_parser.parse(use_tool_response.message.content)

ActionReasoningStep(thought='The current language of the user is: 中文. I need to use a tool to help me answer the question.', action='add', action_input={'a': 1, 'b': 1})

In [10]:
# don't use tool case:
output_parser.parse(no_tool_response.message.content)

ResponseReasoningStep(thought='The current language of the user is: 中文。我可以不用其他工具回答。我會用中文回答。', response='1 + 1 = 2', is_streaming=False)

In [11]:
# special case for don't use tool
output_parser.parse('抄下我聯絡電話是香港3345678， 我再重覆一次是香港3345678。你不打來找我無所謂，因為這將是你的損失。 十點後鐘不要打來，因為我睡啦！')

ResponseReasoningStep(thought='(Implicit) I can answer without any more tools!', response='抄下我聯絡電話是香港3345678， 我再重覆一次是香港3345678。你不打來找我無所謂，因為這將是你的損失。 十點後鐘不要打來，因為我睡啦！', is_streaming=False)

In [12]:
# error parse case
try:
    output_parser.parse('Thought: ')
except Exception as e:
    print(ObservationReasoningStep(observation=f"There was an error in parsing my reasoning: {e}"))

observation='There was an error in parsing my reasoning: Could not parse output: Thought: ' return_direct=False
